In [1]:

import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.layers import *
import math
from tensorflow.keras import backend as K
import sys
import cv2
import os


In [2]:
import torch

In [3]:
import pickle

In [4]:
tf.compat.v1.disable_eager_execution()
tf.reset_default_graph()

batch_size = 128
latent_dim = 64
label_size = 1
bbx_size = 4 #change to 6 for oriented bbox
class_size = 10
lr =0.00005
nb_epochs=300
max_num_node = 24

true_node = tf.placeholder(tf.float32, [batch_size, max_num_node, label_size + bbx_size])

true_class = tf.placeholder(tf.float32 , [batch_size, max_num_node,1])

true_classpred = tf.placeholder(tf.float32 , [batch_size, max_num_node,1])

true_edge = tf.placeholder(tf.float32 , [batch_size, max_num_node, max_num_node])

class_vec = tf.placeholder(tf.float32 , [batch_size, class_size])
class_vecpred = tf.placeholder(tf.float32 , [batch_size, class_size])

kl_weight = tf.placeholder(tf.float32)

dim_vec = tf.placeholder(tf.float32 , [batch_size, 2])
keep_prob = tf.placeholder(tf.float32)
area_weights =  tf.placeholder(tf.float32 , [batch_size, max_num_node,1])

def init_weights(shape):
    init = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)

def init_bias(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

def conv2d(x,W):
    # x ----> [batch , height , width , channels]
    # w ----> [filter height , filter width , channel in , channel out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='VALID')

def max_pool_2b2(x):
    # x ----> [batch , height , width , channels]
    return tf.nn.max_pool(x , ksize = [1,2,2,1] , strides= [1,2,2,1],padding='SAME')

def conv_layer(input_x , shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

def calc_num_wts():    
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print("Total number of trainable parameters:", total_parameters)

def sampling(z_mean, z_log_var):
    epsilon = tf.random_normal(tf.shape(z_log_var), name="epsilon")
    return z_mean + epsilon * tf.exp(z_log_var)

def GCLayer(E, X, out_dims):
    W = tf.Variable(np.random.normal(0, 0.2, size=[X.shape[-1], out_dims]), dtype=tf.float32, name='W')
    W = tf.tile(tf.expand_dims(W, axis=0), [tf.shape(X)[0], 1, 1])
    
    T = tf.convert_to_tensor(tf.keras.backend.sum(E, axis=-1))
    T = tf.convert_to_tensor(tf.linalg.diag(T))
    T = tf.matrix_inverse(T)

    EX = tf.matmul(T, E)
    EX1 = tf.matmul(EX, X)
    EXW = tf.matmul(EX1, W)
    X_out = tf.nn.relu(EXW)

    return X_out

def encoder(E, X_DATA, latent_dim, class_info):
    X1 = GCLayer(E, X_DATA, 32)
    X2 = GCLayer(E, X1, 16)
    
    BOXES =X_DATA[:,:, label_size::]
    BOXES =tf.nn.relu(tf.layers.dense(BOXES, 16))
    
    LABELS =X_DATA[:,:, :label_size]
    LABELS =tf.nn.relu(tf.layers.dense(LABELS, 16))

    MIX = tf.keras.layers.Add()([BOXES, LABELS])

    MIX_FLAT = tf.reshape(MIX, [-1, MIX.shape[1]*MIX.shape[2]])
    MIX_DENSE = tf.nn.relu(tf.layers.dense(MIX_FLAT, 128))

    
    X2_f = tf.reshape(X2, [-1, X2.shape[1]*X2.shape[2]])
    X2_f = tf.keras.layers.concatenate([class_info, X2_f], axis = -1)
    X3 = tf.nn.relu(tf.layers.dense(X2_f, 128))
    X4 = tf.keras.layers.Add()([MIX_DENSE, X3])
    X5 = tf.nn.relu(tf.layers.dense(X4, 128))
    X5 = tf.nn.relu(tf.layers.dense(X5, 128))
        
    z_mean = tf.nn.relu(tf.layers.dense(X5, latent_dim))
    z_logvar = tf.nn.relu(tf.layers.dense(X5, latent_dim))
    
    return z_mean, z_logvar

def decoder(z_latent, num_nodes):
    x1 = tf.nn.relu(tf.layers.dense(z_latent, 128))
    x3d = tf.nn.relu(tf.layers.dense(x1, 128))
    x3 = tf.nn.relu(tf.layers.dense(x3d, 128))
    
    x_bbx = tf.nn.sigmoid(tf.layers.dense(x3, num_nodes*(bbx_size)))
    x_bbx = tf.reshape(x_bbx, [-1, num_nodes, bbx_size])
    
    x_lbl = tf.nn.sigmoid(tf.layers.dense(x3, num_nodes*(label_size)))
    x_lbl = tf.reshape(x_lbl, [-1, num_nodes, label_size])
    
    x_edge = tf.nn.sigmoid(tf.layers.dense(x3, num_nodes*num_nodes))
    x_edge = tf.reshape(x_edge, [-1, num_nodes, num_nodes])
    
    class_ = tf.nn.softmax(tf.layers.dense(x3, class_size))
    
    return x_bbx, x_lbl, x_edge, class_

def conditioning(condition, out_dims):
    W = tf.Variable(np.random.normal(0, 0.2, size=[condition.shape[-1], out_dims]), dtype=tf.float32)
    B = tf.Variable(tf.constant(0.1, shape=[out_dims]))
    
    W = tf.tile(tf.expand_dims(W, axis=0), [tf.shape(condition)[0], 1, 1])
    
    weighted_condition = tf.matmul(condition, W) + B
    return weighted_condition

def condition_z(condition):
    weighted_condition = conditioning(condition, 32)
    reshaped_condition = tf.reshape(weighted_condition, [-1, weighted_condition.shape[1]*weighted_condition.shape[2]])
    return tf.nn.relu(tf.layers.dense(reshaped_condition, 64))

def AutoEncodertf(E, X, latent_dimm, condition, class_condition):
    z_mean, z_logvar = encoder(E, X, latent_dimm, class_condition)
    z_latent = sampling(z_mean, z_logvar)
    condition_ = tf.reshape(condition, (-1, condition.shape[1]*condition.shape[2]))
    
    conditioned_z = tf.keras.layers.concatenate([condition_, z_latent], axis = -1)
    
    conditioned_z = tf.keras.layers.concatenate([class_condition, conditioned_z], axis = -1)
    
    print("Output shape of condition: ", conditioned_z.shape)
    node_box_r, node_cls_r, E_recons, class_ = decoder(conditioned_z, max_num_node)
    print("Output Shapes of Decoder- bbx_recons: {}, lbl_recons: {}, E_recons: {}".format(node_box_r.shape, node_cls_r.shape, E_recons.shape))
    
    return node_box_r, node_cls_r, E_recons, z_latent, z_mean, z_logvar,conditioned_z,class_

node_box_r, node_cls_r, edge_r, z_latent, z_mean, z_logvar, conditioned_z, class_= AutoEncoder(true_edge, true_node, latent_dim,  true_class, class_vec)

node_cls_t = true_node[:, :, :label_size]
node_box_t = true_node[:, :, label_size:]

def frange_cycle_linear(n_iter, start=0.0, stop=1.0,  n_cycle=4, ratio=0.5):
    L = np.ones(n_iter) * stop
    period = n_iter/n_cycle
    step = (stop-start)/(period*ratio)

    for c in range(n_cycle):
        v, i = start, 0
        while v <= stop and (int(i+c*period) < n_iter):
            L[int(i+c*period)] = v
            v += step
            i += 1
    return L

def graph_loss(A_true, A_pred):    
    diag_elem = tf.zeros(A_pred.shape[0:-1])
    diag_elem = tf.cast(diag_elem, tf.float32)
    
    true_nodes = tf.linalg.diag_part(A_true)
    pred_nodes = tf.linalg.diag_part(A_pred)
    
    true_edges = tf.matrix_set_diag(A_true, diag_elem, name='true_edges')
    pred_edges = tf.matrix_set_diag(A_pred, diag_elem, name='pred_edges')

    node_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=true_nodes, logits=pred_nodes))
    edge_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=true_edges, logits=pred_edges))
    
    k = A_true.shape[1]
    k = tf.cast(k, dtype=tf.float32)
    
    total_loss = (node_loss / k) + (edge_loss / (k*(k-1)))
    
    return total_loss

def upper_triangle(mat):
    t = tf.matrix_band_part(mat, 0, -1)
    diag_elem = tf.zeros(t.shape[0:-1])
    diag_elem = tf.cast(diag_elem, tf.float32)
    ut_mat = tf.matrix_set_diag(t, diag_elem, name='ut_mat')
    return ut_mat

def graph_loss_ut(A_true, A_pred):
    A_true_ut = upper_triangle(A_true)
    A_pred_ut = upper_triangle(A_pred)
    
    loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=A_true_ut, logits=A_pred_ut))
    k = A_true.shape[1]
    k = tf.cast(k, dtype=tf.float32)
    nb_edges = k*k/2 - k
    loss = loss / (nb_edges)
    return loss

def smooth_l1_loss(y_true, y_pred):
  """Implements Smooth-L1 loss.
  y_true and y_pred are typically: [N, 4], but could be any shape.
  """
  diff = K.abs(y_true - y_pred)
  less_than_one = K.cast(K.less(diff, 0.01), "float32")
  loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.005)
  return loss

def box_loss(tru_box, gen_box):
    gen_box = ((gen_box)*(tru_box != 0))
    tru_box = ((tru_box)*(tru_box != 0))
    sum_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box)), tf.float32)
    num_r = tf.dtypes.cast(tf.math.count_nonzero(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box), axis=-1)), tf.float32)
    return (sum_r/(num_r+1))

def box_loss_wt(tru_box, gen_box,area_weights):
    gen_box = ((gen_box)*(tru_box != 0))
    tru_box = ((tru_box)*(tru_box != 0))
    sum_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box)**tf.squeeze(area_weights)), tf.float32)
    num_r = tf.dtypes.cast(tf.math.count_nonzero(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box), axis=-1)), tf.float32)
    return (sum_r/(num_r+1))

def area(boxlist):
    x_min, y_min, x_max, y_max = tf.split(
        value=boxlist, num_or_size_splits=4, axis=-1)
    return (y_max - y_min+ 1e-10) * (x_max - x_min + 1e-10)

def aspect_ratio(boxlist):
    x_min, y_min, x_max, y_max = tf.split(
        value=boxlist, num_or_size_splits=4, axis=-1)
    return (y_max - y_min + 1e-10 ) / (x_max - x_min + 1e-10)

def iou(target,  output):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))

    x1g, y1g, x2g, y2g = tf.split(value=target, num_or_size_splits=4, axis=-1)
    x1, y1, x2, y2 = tf.split(value=output, num_or_size_splits=4, axis=-1)
    
    ###iou###
    xA = tf.maximum(x1g, x1)
    yA = tf.maximum(y1g, y1)
    xB = tf.minimum(x2g, x2)
    yB = tf.minimum(y2g, y2)
    
    interArea = tf.maximum(0.0, (xB - xA + 1)) * tf.maximum(0.0, yB - yA + 1)
    boxAArea = (x2g - x1g +1) * (y2g - y1g +1)
    boxBArea = (x2 - x1 +1) * (y2 - y1 +1)
    iouk = interArea / (boxAArea + boxBArea - interArea)
    
    return iouk

def iou_obb(target,  output):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))

    x1g, y1g, x2g, y2g,dg = tf.split(value=target, num_or_size_splits=5, axis=-1)
    x1, y1, x2, y2,d = tf.split(value=output, num_or_size_splits=5, axis=-1)
    ###iou###
    xA = tf.maximum(x1g, x1)
    yA = tf.maximum(y1g, y1)
    xB = tf.minimum(x2g, x2)
    yB = tf.minimum(y2g, y2)
    interArea = tf.maximum(0.0, (xB - xA + 1)) * tf.maximum(0.0, yB - yA + 1)
    boxAArea = (x2g - x1g +1) * (y2g - y1g +1)
    boxBArea = (x2 - x1 +1) * (y2 - y1 +1)
    iouk = interArea / (boxAArea + boxBArea - interArea)
    
    return iouk

def pair_loss(target,  output):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))

    output_unstacked = tf.unstack(output,num=None,axis=-2)
    target_unstacked = tf.unstack(target,num=None,axis=-2)

    pairwise_iou_output = []
    pairwise_iou_target = []
    
    for ii in range(len(target_unstacked)):
        jj = ii
        while jj<(len(target_unstacked)):
            pairwise_iou_output.append((tf.keras.losses.MSE(output_unstacked[ii] , output_unstacked[jj])))
            pairwise_iou_target.append((tf.keras.losses.MSE(target_unstacked[ii] , target_unstacked[jj])))
            jj = jj + 1

    pairwise_iou_output = tf.convert_to_tensor(pairwise_iou_output,dtype=tf.float32)
    pairwise_iou_target = tf.convert_to_tensor(pairwise_iou_target,dtype=tf.float32)
    all_loss_sum = tf.reduce_sum(tf.keras.losses.MSE(pairwise_iou_target, pairwise_iou_output))
    total_non_zero = tf.dtypes.cast(tf.math.count_nonzero(tf.reduce_sum(tf.keras.losses.MSE(pairwise_iou_target, pairwise_iou_output),  axis = -1)),  dtype=tf.float32)
    return all_loss_sum/(total_non_zero+1)

def pair_loss_wt(target,  output,area_weights):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))
    
    output_unstacked = tf.unstack(output,num=None,axis=-2)
    target_unstacked = tf.unstack(target,num=None,axis=-2)
    area_weights_unstacked = tf.unstack(area_weights,num=None,axis=-2)
    pairwise_iou_output = []
    pairwise_iou_target = []
    for ii in range(len(target_unstacked)):
        jj = ii
        while jj<(len(target_unstacked)):
            pairwise_iou_output.append((tf.keras.losses.MSE(output_unstacked[ii] ,
                                                            output_unstacked[jj]))*area_weights_unstacked[ii])
            pairwise_iou_target.append((tf.keras.losses.MSE(target_unstacked[ii] , 
                                                            target_unstacked[jj]))*area_weights_unstacked[ii])
            jj = jj + 1

    pairwise_iou_output = tf.convert_to_tensor(pairwise_iou_output,dtype=tf.float32)
    pairwise_iou_target = tf.convert_to_tensor(pairwise_iou_target,dtype=tf.float32)
    all_loss_sum = tf.reduce_sum(tf.keras.losses.MSE(pairwise_iou_target, pairwise_iou_output))
    total_non_zero = tf.dtypes.cast(tf.math.count_nonzero(tf.reduce_sum(tf.keras.losses.MSE(pairwise_iou_target, pairwise_iou_output),  axis = -1)),  dtype=tf.float32)
    return all_loss_sum/(total_non_zero+1)

def compute_ciou(target,  output):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))

    x1g, y1g, x2g, y2g = tf.split(value=target, num_or_size_splits=4, axis=-1)
    x1, y1, x2, y2 = tf.split(value=output, num_or_size_splits=4, axis=-1)
    
    w_pred = x2 - x1
    h_pred = y2 - y1
    w_gt = x2g - x1g
    h_gt = y2g - y1g

    x_center = (x2 + x1) / 2
    y_center = (y2 + y1) / 2
    x_center_g = (x1g + x2g) / 2
    y_center_g = (y1g + y2g) / 2

    xc1 = tf.minimum(x1, x1g)
    yc1 = tf.minimum(y1, y1g)
    xc2 = tf.maximum(x2, x2g)
    yc2 = tf.maximum(y2, y2g)
    
    ###iou###
    xA = tf.maximum(x1g, x1)
    yA = tf.maximum(y1g, y1)
    xB = tf.minimum(x2g, x2)
    yB = tf.minimum(y2g, y2)
    interArea = tf.maximum(0.0, (xB - xA + 1)) * tf.maximum(0.0, yB - yA + 1)
    boxAArea = (x2g - x1g +1) * (y2g - y1g +1)
    boxBArea = (x2 - x1 +1) * (y2 - y1 +1)
    iouk = interArea / (boxAArea + boxBArea - interArea)
    ciouk = -tf.log(iouk)
    return tf.reduce_mean(ciouk)

    
def compute_ciou_wt(target,  output,area_weights):

    output = ((output)*(target != 0))
    target = ((target)*(target != 0))

    x1g, y1g, x2g, y2g = tf.split(value=target, num_or_size_splits=4, axis=-1)
    x1, y1, x2, y2 = tf.split(value=output, num_or_size_splits=4, axis=-1)
    
    w_pred = x2 - x1
    h_pred = y2 - y1
    w_gt = x2g - x1g
    h_gt = y2g - y1g

    x_center = (x2 + x1) / 2
    y_center = (y2 + y1) / 2
    x_center_g = (x1g + x2g) / 2
    y_center_g = (y1g + y2g) / 2

    xc1 = tf.minimum(x1, x1g)
    yc1 = tf.minimum(y1, y1g)
    xc2 = tf.maximum(x2, x2g)
    yc2 = tf.maximum(y2, y2g)
    
    ###iou###
    xA = tf.maximum(x1g, x1)
    yA = tf.maximum(y1g, y1)
    xB = tf.minimum(x2g, x2)
    yB = tf.minimum(y2g, y2)
    interArea = tf.maximum(0.0, (xB - xA + 1)) * tf.maximum(0.0, yB - yA + 1)
    boxAArea = (x2g - x1g +1) * (y2g - y1g +1)
    boxBArea = (x2 - x1 +1) * (y2 - y1 +1)
    iouk = interArea / (boxAArea + boxBArea - interArea)
    ciouk = -tf.log(iouk)*area_weights
    return tf.reduce_mean(ciouk)


kl_loss = tf.reduce_mean(0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(tf.exp(z_logvar)) - 2*(z_logvar) - 1, axis=1))
adj_loss = tf.reduce_mean(tf.keras.backend.binary_crossentropy(true_edge, edge_r))
bbox_loss = (compute_ciou(node_box_t, node_box_r)) + box_loss(node_box_t, node_box_r) + pair_loss(node_box_t, node_box_r)

cls_loss = tf.reduce_mean(tf.keras.backend.binary_crossentropy(node_cls_t,node_cls_r))
class_vvv = tf.reduce_mean(tf.keras.backend.categorical_crossentropy(class_vec, class_))


reconstuction_loss = (bbox_loss + cls_loss + adj_loss + class_vvv)*24*5 + kl_weight*kl_loss



Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Output shape of condition:  (128, 98)
Output Shapes of Decoder- bbx_recons: (128, 24, 4), lbl_recons: (128, 24, 1), E_recons: (128, 24, 24)


In [5]:
outfile = 'D:/meronym_data/X_train_val.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/class_v_val.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/adj_train_val.np'
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)

In [210]:
def AutoEncodertf(E, X, latent_dimm, condition, class_condition):
    z_mean, z_logvar = encoder(E, X, latent_dimm, class_condition)
    z_latent = sampling(z_mean, z_logvar)
    condition_ = tf.reshape(condition, (-1, condition.shape[1]*condition.shape[2]))
    
    conditioned_z = tf.keras.layers.concatenate([condition_, z_latent], axis = -1)
    
    conditioned_z = tf.keras.layers.concatenate([class_condition, conditioned_z], axis = -1)
    
    print("Output shape of condition: ", conditioned_z.shape)
    node_box_r, node_cls_r, E_recons, class_ = decoder(conditioned_z, max_num_node)
    print("Output Shapes of Decoder- bbx_recons: {}, lbl_recons: {}, E_recons: {}".format(node_box_r.shape, node_cls_r.shape, E_recons.shape))
    
    return node_box_r, node_cls_r, E_recons, z_latent, z_mean, z_logvar,conditioned_z,class_

with tf.Session() as sess:
    tf.random.set_random_seed(100)
    embed = AutoEncodertf(adj_train_val[:1],X_train_val[:1],64, X_train_val[:1,:,:1],class_v_val[:1])
    sess.run(tf.global_variables_initializer())
    embedding = sess.run(embed)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [245]:
node_box_r, node_cls_r, E_recons, z_latent, z_mean, z_logvar,conditioned_z,class_ = embedding

In [219]:
sess = tf.Session()
sess.run(pair_loss(X_train_val[:1,:,1:].astype('float32'),node_box_r))

0.14521512

In [228]:
sess = tf.Session()
sess.run(pair_loss(X_train_val[:1,:,1:].astype('float32'),node_data_pred.detach().cpu().numpy().astype('float32')))

0.037283994

In [268]:
sess = tf.Session()
sess.run(tf.reduce_mean(tf.keras.backend.binary_crossentropy(adj_train_val[:1], E_recons)))

0.69942504

In [252]:
losses.kl_loss(torch.from_numpy(z_mean).cuda(),torch.from_numpy(z_logvar).cuda())

tensor(0.5386, device='cuda:0')

In [262]:
torch.set_printoptions(precision=8)

In [248]:
losses.kl_loss(torch.from_numpy(z_mean), torch.from_numpy(z_logvar))

tensor(0.5386)

In [270]:
losses.adj_loss(torch.from_numpy(E_recons).cuda(),adj_true,batch,num_nodes=24)

tensor(0.69942522, device='cuda:0')

In [209]:
sess = tf.Session()
gen_box = node_box_r#node_data_pred.detach().cpu().numpy().astype('float32')
tru_box = X_train_val[:1,:,1:].astype('float32')
gen_box = ((gen_box)*(tru_box != 0))
tru_box = ((tru_box)*(tru_box != 0))
sum_r = tf.dtypes.cast(tf.keras.losses.MSE(tru_box, gen_box), tf.float32)
num_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box), axis=-1), tf.float32)
loss = sum_r/(num_r+1)
sess.run(loss)




array([[0.0296247 , 0.00390405, 0.        , 0.00771168, 0.        ,
        0.01131818, 0.        , 0.        , 0.01071849, 0.00561339,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]], dtype=float32)

In [217]:
bbox_loss(torch.from_numpy(node_box_r).cuda(),torch.from_numpy(X_train_val[:1,:,1:]).cuda())

torch.Size([1, 24, 1])
torch.Size([1, 4])
torch.Size([300])


(tensor(0.0565, device='cuda:0', dtype=torch.float64),
 tensor(0.0170, device='cuda:0', dtype=torch.float64),
 tensor(0.1452, device='cuda:0', dtype=torch.float64))

In [224]:
bbox_loss(node_data_pred,torch.from_numpy(X_train_val[:1,:,1:]).cuda())

torch.Size([1, 24, 1])
torch.Size([1, 4])
torch.Size([300])


(tensor(0.0555, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0189, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(0.0373, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>))

In [215]:
def box_loss(tru_box, gen_box):
    gen_box = ((gen_box)*(tru_box != 0))
    tru_box = ((tru_box)*(tru_box != 0))
    sum_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box)), tf.float32)
    num_r = tf.dtypes.cast(tf.math.count_nonzero(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box), axis=-1)), tf.float32)
    
    return sum_r/(num_r+1)

In [9]:
from Encoder import Encoder
from AutoEncoder import AutoEncoder

In [10]:
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse, to_dense_adj
import losses
from torch_geometric.data import DataLoader

In [11]:
import torch.nn.functional as F


In [220]:
val_list = []
for idx, batch in enumerate(zip(X_train_val[:1],
                                class_v_val[:1], 
                                adj_train_val[:1])):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).float())
    val_list.append(Data(x = torch.from_numpy(batch[0]).float(),
                         y = torch.from_numpy(batch[1]).float(),
                         edge_index = edge_index
                                )
                     )
batch_val_loader = DataLoader(val_list, batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [255]:
latent_dims = 64
batch_size = 64
num_nodes = 24
bbx_size = 4
num_classes = 10
label_shape = 1
nb_epochs = 50
learning_rate = 0.0001
hidden1 = 32
hidden2 = 16
hidden3 = 128
vae = AutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3)
vae.cuda()
for val_data in batch_val_loader:
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    adj_true = val_data.edge_index
    batch = val_data.batch


    output = vae(adj_true, node_data_true, label_true , class_true)
    node_data_pred, label_pred, adj_pred, class_pred, z_mean, z_logvar = output

C:\Users\user\Documents\Workspace\MeronymNet-PyTorch\src\components\Decoder.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  class_pred = self.act2(self.dense_cls(x))


In [223]:
sess = tf.Session()
gen_box = node_data_pred.detach().cpu().numpy().astype('float32')
tru_box = X_train_val[:2,:,1:].astype('float32')
gen_box = ((gen_box)*(tru_box != 0))
tru_box = ((tru_box)*(tru_box != 0))
sum_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box)), tf.float32)
num_r = tf.dtypes.cast(tf.reduce_sum(tf.keras.losses.MSE(tru_box, gen_box), axis=-1), tf.float32)
loss = sum_r/(num_r+1)
sess.run(loss)

array([0.4468577 , 0.32522985], dtype=float32)

In [152]:
node_data_pred.shape

torch.Size([2, 24, 4])

In [153]:
sess = tf.Session()
gen_box = node_data_pred.detach().cpu().numpy().astype('float32')
true_box = X_train_val[:2,:,1:].astype('float32')
sess.run(compute_ciou(true_box,gen_box))

0.14010613

In [15]:
2.4+2.8+1.6

6.799999999999999

In [17]:
tp.shape

torch.Size([1, 24])

In [149]:
bbox_loss(node_data_pred, node_data_true[:,1:])

torch.Size([2, 24, 1])
torch.Size([2, 4])
torch.Size([300])


(tensor(0.1544, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1592, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(0.4687, device='cuda:0', grad_fn=<DivBackward0>))

In [148]:
def bbox_loss(pred_box, true_box):
    
    # IOU loss
    true_box = torch.reshape(true_box,pred_box.shape)
    mask = torch.where(torch.sum(true_box,dim=-1,keepdim=True)!=0,1.0,0.0)
    pred_box = mask*pred_box
    x1g, y1g, x2g, y2g = torch.tensor_split(true_box, 4, dim=-1)
    x1, y1, x2, y2 = torch.tensor_split(pred_box, 4, dim=-1)
    
    xA = torch.maximum(x1g, x1)
    yA = torch.maximum(y1g, y1)
    xB = torch.minimum(x2g, x2)
    yB = torch.minimum(y2g, y2)
    
    interArea = torch.maximum(torch.tensor([0.0]).cuda(),
                              (xB - xA + 1)) * torch.maximum(torch.tensor([0.0]).cuda(),
                                                         yB - yA + 1)
    boxAArea = (x2g - x1g + 1) * (y2g - y1g + 1)
    boxBArea = (x2 - x1 + 1) * (y2 - y1 + 1)
    iouk = (interArea) / (boxAArea + boxBArea - interArea + 1e-06)
    iou_loss = -torch.log(iouk + 1e-6)
    iou_loss = torch.mean(iou_loss)#/torch.sum(mask)
    
    # Box regression loss
    reg_loss = F.mse_loss(pred_box, true_box, reduction='none')
    reg_loss = torch.mean(reg_loss,dim = -1)
    reg_loss = torch.sum(reg_loss,dim = -1)
    total_non_zero = torch.count_nonzero(reg_loss)
    reg_loss = torch.sum(reg_loss)/(total_non_zero+1)
    
    # Pairwise box regression loss
    pair_mse_true = []
    pair_mse_pred = []
    true_unstacked = torch.unbind(true_box,dim=-2)
    pred_unstacked = torch.unbind(pred_box,dim=-2)
    
    for i in range(len(true_unstacked)):
        
        for j in range(i, len(true_unstacked)):
            pair_mse_true.append(F.mse_loss(true_unstacked[i],true_unstacked[j]))
            pair_mse_pred.append(F.mse_loss(pred_unstacked[i],pred_unstacked[j]))
        
    
    pair_loss = F.mse_loss(torch.stack(pair_mse_pred), 
                           torch.stack(pair_mse_true),
                           reduction='none',
                           )
    total_non_zero = torch.count_nonzero(torch.sum(pair_loss,dim=-1))
    pair_loss = torch.sum(pair_loss)/(total_non_zero+1)
    
    return iou_loss , reg_loss , pair_loss
